<a href="https://colab.research.google.com/github/palswayam5/AI_generated_Images_Classifier/blob/master/ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense,LayerNormalization,Dropout,Add,Embedding,MultiHeadAttention,Input,Flatten
from tensorflow.keras import Model
from tensorflow.image import extract_patches
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Dataset/test.csv')

In [3]:
df.head()

,labels,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,...,f_1190,f_1191,f_1192,f_1193,f_1194,f_1195,f_1196,f_1197,f_1198,f_1199
0,0,-2.033875,0.978446,-0.142131,-0.177117,-1.470684,1.669562,-0.196530,-0.125239,-0.452284,...,-1.111266,0.716084,0.060039,0.301279,-1.174846,-1.076498,-0.069452,-0.604012,-2.179176,0.558003
1,1,-0.348835,0.294815,-0.557577,-2.020773,-1.234715,1.633930,-1.680658,-0.358146,0.166122,...,0.735240,0.829781,1.521941,1.347946,0.754505,1.330642,-0.754453,0.582956,0.252671,1.495870
2,1,0.113248,-0.607726,-0.947791,0.830851,0.998291,0.498321,-1.493958,0.789572,-1.311018,...,0.104698,0.616189,-1.035953,2.111387,-0.984415,1.148076,-1.433554,0.243372,0.170083,1.274795
3,0,1.223321,-0.479048,-1.925789,1.680377,0.021840,-1.453307,0.605559,-0.019024,1.065448,...,0.360237,-1.957863,-0.123384,1.505329,0.660290,-1.769443,-0.547756,-0.568122,0.244645,0.982116
4,0,0.160109,0.422684,-0.308029,0.227744,0.432854,0.608348,0.193832,1.035091,-0.538868,...,0.416629,1.441766,0.212572,-0.994721,1.143999,-2.166923,-1.199248,-1.028636,0.752791,0.317169


In [4]:
X = df.drop('labels',axis = 1).to_numpy()
y = df['labels'].to_numpy()
X = X.reshape(-1,20,20,3)
y = y.reshape(-1,1)

In [5]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = 0.2,shuffle = True)

In [6]:
print(X_train.shape,y_train.shape,X_val.shape,y_val.shape)

(4200, 20, 20, 3) (4200, 1) (1050, 20, 20, 3) (1050, 1)


Variables

In [7]:
num_layers = 8 #repeating layers in a vision transformer
input_shape = (20,20,3)
patch_size = 4
final_hidden_units = [2048,1024];
dropout = 0.2
head = 4
dim = 64
num_patches = (input_shape[0]//patch_size)**2 # here 25
num_classes = 2
transformer_units = [dim * 2,dim]

Multi Layer Perceptron

In [27]:
def mlp(x,hidden_units,dropout_rate):   #multi_layer_perceptron
   for units in hidden_units:
    x =  Dense(units = units)(x)
    x = Dropout(dropout_rate)(x)
   return x

In [13]:
def create_patches(image):
  batch_size = tf.shape(image)[0]
  patches = extract_patches(
      images = image,
      sizes = [1,patch_size,patch_size,1],
      strides = [1,patch_size,patch_size,1],
      padding = "VALID",
      rates = [1,1,1,1],
  )
  patch_dims = patches.shape[-1]
  patches = tf.reshape(patches, [batch_size, -1, patch_dims])
  return patches

In [18]:
def encode_patches(patches):
  positions = tf.range(start=0, limit=num_patches, delta=1)
  patches = Dense(units = dim)(patches)
  positions = Embedding(input_dim = num_patches,output_dim = dim)(positions)
  encoded_patches = patches+positions
  return encoded_patches

In [34]:
def vit_classifier():
  input = Input(shape = input_shape)
  patches = create_patches(input)
  encoded_patches = encode_patches(patches)

  for i in range(num_layers):
    x = LayerNormalization(epsilon = 1e-6)(encoded_patches)
    x1 = MultiHeadAttention(num_heads = head,key_dim = dim,dropout = dropout)(x,x)
    x2 = Add()([x1,encoded_patches])
    x2 = LayerNormalization(epsilon = 1e-6)(x2)
    x3 = mlp(x2,transformer_units,dropout)
    encoded_patches = Add()([x3,x2])
  x = LayerNormalization(epsilon = 1e-6)(encoded_patches)
  x = Flatten()(x)
  x = Dropout(dropout)(x)
  x = mlp(x,final_hidden_units,dropout)
  y = Dense(units = num_classes,activation = "softmax")(x)
  model = Model(inputs = input,outputs = y)
  return model


In [35]:
model = vit_classifier()

In [37]:
model.compile(optimizer = 'Adam',loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])

In [39]:
model.fit(X_train,y_train,validation_data = (X_val,y_val),epochs = 10)

Epoch 1/10
132/132 [==============================] - 7s 50ms/step - loss: 2.2405 - accuracy: 0.7293 - val_loss: 0.5484 - val_accuracy: 0.7629
Epoch 2/10
132/132 [==============================] - 6s 43ms/step - loss: 0.4780 - accuracy: 0.7779 - val_loss: 0.4253 - val_accuracy: 0.7914
Epoch 3/10
132/132 [==============================] - 6s 43ms/step - loss: 0.4359 - accuracy: 0.8048 - val_loss: 0.4229 - val_accuracy: 0.7733
Epoch 4/10
132/132 [==============================] - 5s 37ms/step - loss: 0.4515 - accuracy: 0.7940 - val_loss: 0.5077 - val_accuracy: 0.7943
Epoch 5/10
132/132 [==============================] - 6s 47ms/step - loss: 0.4561 - accuracy: 0.8071 - val_loss: 0.4450 - val_accuracy: 0.8143
Epoch 6/10
132/132 [==============================] - 5s 42ms/step - loss: 0.4585 - accuracy: 0.8055 - val_loss: 0.5402 - val_accuracy: 0.7410
Epoch 7/10
132/132 [==============================] - 7s 50ms/step - loss: 0.4653 - accuracy: 0.8002 - val_loss: 0.5866 - val_accuracy: 0.8124

In [42]:
y_val_predict = model.predict(X_val)
y_val_predict

33/33 [==============================] - 0s 12ms/step


array([[0.33100998, 0.66899   ],
       [0.9569489 , 0.04305117],
       [0.41433623, 0.5856638 ],
       ...,
       [0.9632544 , 0.03674556],
       [0.4446971 , 0.55530286],
       [0.70727617, 0.29272383]], dtype=float32)

In [44]:
y_val_fa = np.zeros([len(y_val_predict)])
for i in range(len(y_val_predict)):
  if(y_val_predict[i][0]>y_val_predict[i][1]):
    y_val_fa[i] = 0
  else:
    y_val_fa[i] = 1
y_val_fa


array([1., 0., 1., ..., 0., 1., 0.])

In [45]:
y_val_fa = y_val_fa.reshape(-1,1)
f1_score(y_val,y_val_fa)

0.6308724832214765

In [46]:
df_test.head()

,id,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,...,f_1190,f_1191,f_1192,f_1193,f_1194,f_1195,f_1196,f_1197,f_1198,f_1199
0,1,-3.388242,0.868285,-0.427619,-0.678964,-1.625735,0.262761,1.243040,1.537751,-0.352028,...,-0.776403,-0.662884,-0.257091,-1.168413,0.223260,-0.482520,-0.085453,-0.382265,-0.539349,-1.682404
1,2,-0.496920,0.952381,0.989040,0.451422,0.513516,-0.099658,-1.124326,0.729430,-0.216224,...,0.379635,-1.760084,1.125450,-0.328047,-0.880305,-1.257607,0.964312,2.021104,0.655021,-0.423029
2,3,1.128369,-0.537951,2.544358,1.165254,-1.904994,0.776961,-0.495768,0.060111,-1.418468,...,1.165254,-1.373589,-0.483701,-0.964782,-0.869555,0.066040,-0.444567,-0.531935,-0.878660,1.099488
3,4,0.051253,1.746814,0.681177,1.844524,-0.327977,1.226839,-0.085519,0.379008,-1.003667,...,-0.442288,-2.794472,-0.763468,-0.789832,-0.113209,-2.703150,-2.058728,1.070627,-0.458045,-0.435825
4,5,1.423209,-0.983594,-1.694170,1.197507,1.044211,0.518777,-0.298612,-0.365174,0.738447,...,-2.624450,-3.200223,0.711422,-0.190394,0.337224,-1.656639,0.707360,-0.562290,1.471181,-0.192000


In [48]:
X_test = df_test.drop('id',axis = 1).to_numpy()
id = df_test['id'].to_numpy()
X_test = X_test.reshape(-1,20,20,3)

In [49]:
y_test = model.predict(X_test)
y_fa = np.zeros([len(y_test)])
for i in range(len(y_test)):
  if(y_test[i][0]>y_test[i][1]):
    y_fa[i] = 0
  else:
    y_fa[i] = 1
y_fa

71/71 [==============================] - 1s 13ms/step


array([1., 0., 1., ..., 1., 0., 1.])

In [55]:
y_fa.shape

(2250,)

In [57]:
id = id.reshape(-1,1)
y_fa = y_fa.reshape(-1,1)
print(id.shape,y_fa.shape)

(2250, 1) (2250, 1)


In [59]:
temp = np.concatenate([id,y_fa],axis = 1)
temp.shape

(2250, 2)

In [61]:
submission = pd.DataFrame(temp,columns = ['id','labels'])

In [62]:
submission.to_csv('vit.csv')